In [3]:
import os
import shutil
from extlist import *
import time
import argparse
import datetime
from openai import OpenAI
import importlib.resources as pkg_resources


class PCSorter:
    """
    File/Folder cleaner.
    :param x: restore file name. string
    :return: none
    """


    files = []
    sub_folders = []
    sub_files = []
    tmp_list = []
    dir_containing_files = []


    def __init__(self, model, directory, is_backup):

        self.is_backup = is_backup
        self.directory = directory
        self.model = model
        self.client = OpenAI(api_key=os.environ['OPENAI_API_KEY'])
        self.chatbot = ""
        with open("./data/chatbot.txt") as file:
            self.chatbot = file.read()

    def _restore(self):
        """
        Restore previous cleaning session
        :return: none
        """
        try:
            with open(self.resetname, "r") as f:
                print("Opening restore file...")
                lines = f.readlines()
                for i in lines:
                    restore_dir = i
                    restore_dir = restore_dir.split("###")

                    program_name = r"{}".format(restore_dir[0].replace("\n", ""))
                    restore_path = r"{}".format(restore_dir[1].replace("\n", ""))
                    curry_path = r"{}".format(restore_dir[2].replace("\n", ""))
                    restore_path = os.path.join(restore_path.strip())
                    cur_path = os.path.join(curry_path)
                    if os.path.exists(cur_path):
                        res_path = restore_path.replace(program_name, "")
                        res_path = os.path.join(res_path)
                        if os.path.exists(res_path):
                            shutil.move(cur_path, restore_path)
                        else:
                            os.makedirs(res_path)
                            shutil.move(cur_path, restore_path)
                print("All files have been restored...")
            # Delete old folders after restorting. 
            for i in ext_dir:
                cur_path = os.path.join(self.current_path, i)
                if os.path.isdir(cur_path):
                    shutil.rmtree(cur_path)
                else:
                    pass
                    #print("Couldnt delete directory.")
            os.remove(self.resetname)
            print("All old sort folders have been deleted...")
            time.sleep(2)
        except FileNotFoundError:
            print("The file was not found..")
        except:
            print("Unkown Error.")

    def _backup(self):
        """
        Backup function
        """
        if not os.path.isdir("Backup"):
            os.mkdir("Backup")
        else:
            print("Folder already created")

        current_files = os.listdir(".")
        for i in current_files:
            try:
                shutil.copy(i, "./Backup")
            except:
                print("File {} couldn't be moved. Unkown reason!".format(i))
        time.sleep(2)

    def _searchfiles(self, filename):
        """
        Search files.
        :param x: filename string
        :return: path of file string
        """

        if len(self.full_list) == 0:
            self._listfiles()

        for i in self.full_list:
            x2 = i.split("\\")
            x2 = x2[-1]
            if filename in x2:
                print("We found your file its located, at {}".format(i))
                return i
    def _listfiles(self, include_subdirs=True):
        """
        Scans the specified directory and its subdirectories for files, excluding certain files.
        If include_subdirs is False, only files from the root directory are returned.
        Returns a tuple of two lists: (all_files, all_directories).
        """
        excluded_files = {'cleaner.py', 'extlist.py'}
        all_files = []
        all_directories = []

        for root, dirs, files in os.walk(self.directory):
            if root != os.path.normpath(self.directory) and not include_subdirs:
                continue  # Skip subdirectories
            all_directories.append(root)
            for file in files:
                if file not in excluded_files:
                    all_files.append(os.path.join(root, file))

        return all_files, all_directories


    def _create_sort_folders(self, ext_dict):
            print("Creating folders...")
            for i in ext_dict:
                ext_path_dir = ext_dict[i]
                if os.path.isdir("./{}".format(ext_path_dir)):
                    print("Folder -> {} already exists".format(ext_path_dir))
                    continue
                else:
                    print("Created Folders -> {}".format(ext_path_dir))    
                    os.makedirs("{}".format(ext_path_dir))
            return True


    def _write_restore_file(self, lists):
        try:
            with open(self.resetname, "w") as f:
                for i in lists:
                    f.write(i)
                f.close()
        except:
            print("_write_restore_file ERROR")


    def _sort_files(self, filenames):

        tmp_list = []
        dir_containing_files = []   
        print("Sorting Files...")
        for i in filenames:
            file_ext = os.path.splitext(i)
            for ext_key in extension_paths:               
                if file_ext[1] == ext_key:   
                    spec_dir = extension_paths["{}".format(ext_key)]

                    file_name = i.split("\\")
                    file_name = file_name[-1]

                    file_path = os.path.join(self.current_path, spec_dir, file_name)               
                    path = os.path.join(self.current_path, spec_dir)
                    old_path = os.path.join(i)
                    if not os.path.exists(file_path):
                        tmp_list.append("{}###{}###{} \n".format(file_name, old_path, file_path))
                        time.sleep(0.01)
                        try:
                            shutil.move(i, path)
                        except:
                            #print("Couldnt move the file.")
                            dir_containing_files.append(os.path.join(file_ext[0]))
                        break
        print("Writing files to restore.txt")
        return dir_containing_files, tmp_list

    def _delete_sorted_folders(self, none_sorted, sub_folders):
        for i in sub_folders:
            if i in none_sorted:
                continue
            else:
                try:
                    shutil.rmtree(os.path.join(i))
                except:
                    print("Couldnt delete folder")



    def _clean_dirs(self):
        
        today = time.strftime('%Y-%m-%d', time.localtime())
        today = today.split("-")
        today_days = ((int(today[0]) * 365 ) + (int(today[1]) * 30) + int(today[2])) 

        for i in self.full_list:
            epoch_time = os.path.getatime(i)
            last_accessed = time.strftime('%Y-%m-%d', time.localtime(epoch_time))
            last_accessed = last_accessed.split("-")

            last_accessed_days = ((int(last_accessed[0]) * 365 ) + (int(last_accessed[1]) * 30) + int(last_accessed[2]))

            diff = today_days - last_accessed_days

            # Meaning the file is older then 90 days.
            if diff > 90:
                print("delete file.")

    def _chatgpt(self, user_input, model="gpt-4-1106-preview", temperature=0, max_tokens=500):
        # Prepare messages for the conversation
        formatted_conversation = [{"role": "system", "content": self.chatbot}]
        formatted_conversation.append({"role": "user", "content": user_input})

        # Create the chat completion using the new API method
        completion = self.client.chat.completions.create(
            model=model,
            messages=formatted_conversation,
            temperature=temperature,
            max_tokens=max_tokens
        )

        # Extract the response
        if completion.choices and completion.choices[0].message:
            chat_response = completion.choices[0].message.content
        else:
            chat_response = "No response generated."

        return chat_response    


In [4]:
import os
# class ins
sorter = PCSorter("gpt4", os.getcwd(), True)

In [1]:
files, dirs = sorter._listfiles()
print(files)

exit(0)
# Normalize the base directory path for consistent comparison
normalized_base_dir = os.path.normpath(os.getcwd())

only_root_files = []
for f in files:
    file_dir = os.path.normpath(os.path.dirname(f))
    if file_dir == normalized_base_dir:
        only_root_files.append(f)
    else:
        # Logging to understand why a file is excluded
        print(f"Excluded: {f}, File dir: {file_dir}, Base dir: {normalized_base_dir}")

# If you need just the filenames, not the full path
only_filename = [os.path.basename(f) for f in only_root_files]


NameError: name 'sorter' is not defined

In [24]:
conversation_history = [
    {"role": "user", "content": "Hello, how are you?"},
    {"role": "system", "content": "I am a chatbot designed to assist with a wide range of questions."}
]

sorter._chatgpt(user_input = """Invoice_4.pdf
Invoice_5.pdf
MeetingNotes_1.txt
MeetingNotes_2.txt
MeetingNotes_3.txt
MeetingNotes_4.txt
MeetingNotes_5.txt
Photos—l.jpg
Photos_2.jpg
Photos_3.jpg
Photos_4.jpg
Photos_5.jpg
Projects_l.docx
Projects_2.docx
Projects_3.docx
Projects_4.docx
Project_5.docx
""")

'```json\n{\n  "Invoices": ["Invoice_4.pdf", "Invoice_5.pdf"],\n  "MeetingNotes": ["MeetingNotes_1.txt", "MeetingNotes_2.txt", "MeetingNotes_3.txt", "MeetingNotes_4.txt", "MeetingNotes_5.txt"],\n  "Photos": ["Photos—l.jpg", "Photos_2.jpg", "Photos_3.jpg", "Photos_4.jpg", "Photos_5.jpg"],\n  "Projects": ["Projects_l.docx", "Projects_2.docx", "Projects_3.docx", "Projects_4.docx", "Project_5.docx"]\n}\n```'

In [1]:
import os

# Given file path
file_path = r"C:\Users\chriek01\Documents\GitHub\Desktop-Cleaner\OrganizerPro\main.py"

# Extract the filename
file_name = os.path.basename(file_path)

print(file_name)


main.py


In [9]:
input_str = """input: Hello my name is Christopher EK and I love 
```json\n{\n  "Invoices": ["Invoice_4.pdf", "Invoice_5.pdf"],\n 
"MeetingNotes": ["MeetingNotes_1.txt", "MeetingNotes_2.txt", "MeetingNotes_3.txt", 
"MeetingNotes_4.txt", "MeetingNotes_5.txt"],\n  "Photos": ["Photos—l.jpg", "Photos_2.jpg", 
"Photos_3.jpg", "Photos_4.jpg", "Photos_5.jpg"],\n  "Projects": ["Projects_l.docx", 
"Projects_2.docx", "Projects_3.docx", "Projects_4.docx", "Project_5.docx"]\n}\
"""



In [10]:
import re

def remove_text_before_first_opening_brace_and_after_last_closing_brace(input_str):
    # Find the index of the first '{' character
    start_index = input_str.find('{')
    
    # Find the index of the last '}' character
    end_index = input_str.rfind('}')

    if start_index != -1 and end_index != -1:
        # Slice the string to remove text before the first '{' and after the last '}'
        result_str = input_str[start_index:end_index+1]
        return result_str
    else:
        return "No '{' or '}' characters found in the input string."
        

In [11]:
remove_text_before_first_opening_brace_and_after_last_closing_brace(input_str)

'{\n  "Invoices": ["Invoice_4.pdf", "Invoice_5.pdf"],\n \n"MeetingNotes": ["MeetingNotes_1.txt", "MeetingNotes_2.txt", "MeetingNotes_3.txt", \n"MeetingNotes_4.txt", "MeetingNotes_5.txt"],\n  "Photos": ["Photos—l.jpg", "Photos_2.jpg", \n"Photos_3.jpg", "Photos_4.jpg", "Photos_5.jpg"],\n  "Projects": ["Projects_l.docx", \n"Projects_2.docx", "Projects_3.docx", "Projects_4.docx", "Project_5.docx"]\n}'

In [13]:
import json

# Your JSON data as a string
json_data = '''
{
  "Invoices": ["Invoice_4.pdf", "Invoice_5.pdf"],
  "MeetingNotes": ["MeetingNotes_1.txt", "MeetingNotes_2.txt", "MeetingNotes_3.txt", "MeetingNotes_4.txt", "MeetingNotes_5.txt"],
  "Photos": ["Photos—l.jpg", "Photos_2.jpg", "Photos_3.jpg", "Photos_4.jpg", "Photos_5.jpg"],
  "Projects": ["Projects_l.docx", "Projects_2.docx", "Projects_3.docx", "Projects_4.docx", "Project_5.docx"]
}
'''

# Parse the JSON data into a Python dictionary
data_dict = json.loads(json_data)

# Define a function to print the folder structure
def print_folder_structure(data, indent=""):
    for folder, files in data.items():
        print(indent + f"Folder: {folder}")
        for file in files:
            print(indent + f"  - {file}")

# Print the folder structure
print_folder_structure(data_dict)


Folder: Invoices
  - Invoice_4.pdf
  - Invoice_5.pdf
Folder: MeetingNotes
  - MeetingNotes_1.txt
  - MeetingNotes_2.txt
  - MeetingNotes_3.txt
  - MeetingNotes_4.txt
  - MeetingNotes_5.txt
Folder: Photos
  - Photos—l.jpg
  - Photos_2.jpg
  - Photos_3.jpg
  - Photos_4.jpg
  - Photos_5.jpg
Folder: Projects
  - Projects_l.docx
  - Projects_2.docx
  - Projects_3.docx
  - Projects_4.docx
  - Project_5.docx
